In [1]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pickle
import os.path
import base64
import email
from bs4 import BeautifulSoup
from datetime import datetime, timedelta, timezone

In [2]:
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

In [3]:
def getEmails():
	# Variable creds will store the user access token.
	# If no valid token found, we will create one.
	creds = None

	# The file token.pickle contains the user access token.
	# Check if it exists
	if os.path.exists('token.pickle'):

		# Read the token from the file and store it in the variable creds
		with open('token.pickle', 'rb') as token:
			creds = pickle.load(token)

	# If credentials are not available or are invalid, ask the user to log in.
	if not creds or not creds.valid:
		if creds and creds.expired and creds.refresh_token:
			creds.refresh(Request())
		else:
			flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
			creds = flow.run_local_server(port=0)

		# Save the access token in token.pickle file for the next run
		with open('token.pickle', 'wb') as token:
			pickle.dump(creds, token)

	# Connect to the Gmail API
	service = build('gmail', 'v1', credentials=creds)

	# request a list of all the messages
	# Calculate the timestamp for 24 hours ago
	now = datetime.now(timezone.utc)
	yesterday = now - timedelta(days=1)
	yesterday_timestamp = int(yesterday.timestamp())
	query = f'after:{yesterday_timestamp} (category:primary OR category:updates)'
	result = service.users().messages().list(userId='me', q=query).execute()
	messages = result.get('messages')

    
	# messages is a list of dictionaries where each dictionary contains a message id.
	# iterate through all the messages
	for msg in messages:
		# Get the message from its id
		txt = service.users().messages().get(userId='me', id=msg['id']).execute()
		# Use try-except to avoid any Errors
		try:
			# Get value of 'payload' from dictionary 'txt'
			payload = txt['payload']
			headers = payload['headers']

			# Look for Subject and Sender Email in the headers
			for d in headers:
				if d['name'] == 'Subject':
					subject = d['value']
				if d['name'] == 'From':
					sender = d['value']


            
			# The Body of the message is in Encrypted format. So, we have to decode it.
			# Get the data and decode it with base 64 decoder.
			
			print(payload)

			if 'parts' in txt['payload']:
				data = txt['payload']['parts'][0]['body']['data']
				data = data.replace("-","+").replace("_","/")
				decoded_data = base64.b64decode(data)
			else:
				data = txt['payload']['body']['data']
				data = data.replace("-","+").replace("_","/")
				decoded_data = base64.b64decode(data)
			

			soup = BeautifulSoup(decoded_data , "html.parser")
			body = soup.get_text()


			# Generate a filename
			filename = f"emails/{sender}_{msg['id']}.txt"

			# Write email data to the file
			with open(filename, 'w', encoding='utf-8') as f:
				f.write(f"Subject: {subject}\n")
				f.write(f"From: {sender}\n")
				f.write("Message:\n")
				f.write(body)


			
		except:
			pass


In [218]:
def getEmails():
    # Variable creds will store the user access token.
    # If no valid token found, we will create one.
    creds = None

    # The file token.pickle contains the user access token.
    # Check if it exists
    if os.path.exists('token.pickle'):

        # Read the token from the file and store it in the variable creds
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)

    # If credentials are not available or are invalid, ask the user to log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)

        # Save the access token in token.pickle file for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    # Connect to the Gmail API
    service = build('gmail', 'v1', credentials=creds)

    # Request a list of all the messages
    # Calculate the timestamp for 24 hours ago
    now = datetime.now(timezone.utc)
    yesterday = now - timedelta(days=1)
    yesterday_timestamp = int(yesterday.timestamp())
    query = f'after:{yesterday_timestamp} (category:primary OR category:updates)'
    result = service.users().messages().list(userId='me', q=query).execute()
    messages = result.get('messages')

    # Check if messages are returned
    if not messages:
        print("No new messages found.")
        return

    # Messages is a list of dictionaries where each dictionary contains a message id.
    # Iterate through all the messages
    for msg in messages:
        # Get the message from its id
        txt = service.users().messages().get(userId='me', id=msg['id']).execute()
        # Use try-except to avoid any Errors
        try:
            # Get value of 'payload' from dictionary 'txt'
            payload = txt['payload']
            headers = payload['headers']

            # Look for Subject and Sender Email in the headers
            subject = sender = None
            for d in headers:
                if d['name'] == 'Subject':
                    subject = d['value']
                elif d['name'] == 'From':
                    sender = d['value']

            # The Body of the message is in Encrypted format. So, we have to decode it.
            # Get the data and decode it with base64 decoder.

            print(payload)

            if 'parts' in txt['payload']:
                for part in txt['payload']['parts']:
                    if part['mimeType'] == 'text/html':
                        data = part['body']['data']
                        data = data.replace("-", "+").replace("_", "/")
                        decoded_data = base64.b64decode(data)
                        soup = BeautifulSoup(decoded_data, "html.parser")

                        # Extract links and append to the text
                        for a in soup.find_all('a', href=True):
                            a_text = f"{a.text} ({a['href']})"
                            a.replace_with(a_text)

                        body = soup.get_text()

                    elif part['mimeType'] == 'text/plain':
                        data = part['body']['data']
                        data = data.replace("-", "+").replace("_", "/")
                        decoded_data = base64.b64decode(data)
                        body = decoded_data.decode('utf-8')
            else:
                data = payload['body'].get('data')
                if data:
                    data = data.replace("-", "+").replace("_", "/")
                    decoded_data = base64.b64decode(data)
                    body = decoded_data.decode('utf-8')

            # Generate a filename
            filename = f"emails/{sender}_{msg['id']}.txt"

            # Write email data to the file
            with open(filename, 'w', encoding='utf-8') as f:
                f.write(f"Subject: {subject}\n")
                f.write(f"From: {sender}\n")
                f.write("Message:\n")
                f.write(body)

        except Exception as e:
            print(f"An error occurred: {e}")
            pass

In [219]:
getEmails()

{'partId': '', 'mimeType': 'multipart/alternative', 'filename': '', 'headers': [{'name': 'Delivered-To', 'value': 'vinitmotwani10@gmail.com'}, {'name': 'Received', 'value': 'by 2002:a54:344f:0:b0:26d:144e:76a8 with SMTP id p15csp2010938ect;        Mon, 26 Aug 2024 16:20:31 -0700 (PDT)'}, {'name': 'X-Received', 'value': 'by 2002:a05:6512:39ca:b0:52f:228:cf80 with SMTP id 2adb3069b0e04-534387be63bmr8825533e87.40.1724714430898;        Mon, 26 Aug 2024 16:20:30 -0700 (PDT)'}, {'name': 'ARC-Seal', 'value': 'i=1; a=rsa-sha256; t=1724714430; cv=none;        d=google.com; s=arc-20160816;        b=cKqEcTKzkd6AQ57QWMWuo0Fkm+Rf4fgav+3JhgTXSokvicvjHh88VOrF+Ert5DCFT9         gKColAQEghDye4zAlml6KEE20aBiYdCIO4jsO4iNlYhYBWVx4XXXwgmDZx3vSr1n0tAa         0DCMPTrcj89J6M4pDHPkT2Trv8ERMdFG1nxQzmFL9uO1Mg5fxtL02a/rc4M16fjogTX5         oRsdkFNsWYVzGuyIH9/DW9tkv6ewMFqF3IZlhAPzdtc1zG5VDF/T7P5L6f9IPe61rYiG         pbjoqlE7b9NCm2m6n6MrXhhmdVFTcFSgOpZLb0M/oe9XvReFuVGsVc8O72V4TE3j65Dz         MoMA=='}, {'name': 'A

In [205]:
decoded_data = base64.b64decode("'DQogICAgICAgIDxodG1sPg0KICAgICAgICA8aGVhZD48L2hlYWQ-DQogICAgICAgIDxib2R5Pg0KICAgICAgICA8cD48Yj5Kb2IgcG9zdGluZ3Mgd2VyZSByZW9wZW5lZCE8L2I-PC9wPg0KICAgICAgICA8YnI-DQogICAgICAgIDx0YWJsZT48dHI-PHRoPkNvbXBhbnk8L3RoPjx0aD5Kb2I8L3RoPjwvdHI-DQogICAgICAgICAgICAgICAgPHRyPg0KICAgICAgICAgICAgICAgICAgICA8dGQ-PGEgaHJlZj0naHR0cHM6Ly9jYXBtZXRyby5vcmcvJz5DYXBpdGFsIE1ldHJvPC9hPjwvdGQ-DQogICAgICAgICAgICAgICAgICAgIDx0ZD48YSBocmVmPSdodHRwczovL210bWluYy53ZDEubXl3b3JrZGF5am9icy5jb20vTVRNX0V4dGVybmFsL2pvYi9NTy0tLVN0LUxvdWlzLS0tMTIzOC9VSS1EZXZlbG9wZXItSUlfUkVRMTAwNTAnPlVJIERldmVsb3BlciBJSTwvYT48L3RkPg0KICAgICAgICAgICAgICAgIDwvdHI-DQogICAgICAgICAgICANCiAgICAgICAgICAgICAgICA8dHI-DQogICAgICAgICAgICAgICAgICAgIDx0ZD48YSBocmVmPSdodHRwczovL2NlbnRlbmUuY29tLyc-Q2VudGVuZTwvYT48L3RkPg0KICAgICAgICAgICAgICAgICAgICA8dGQ-PGEgaHJlZj0naHR0cHM6Ly9jZW50ZW5lLndkNS5teXdvcmtkYXlqb2JzLmNvbS9DZW50ZW5lX0V4dGVybmFsL2pvYi9SZW1vdGUtRkwvQnVzaW5lc3MtQW5hbHlzdC1JSV8xNTEwNTI1LTEnPkJ1c2luZXNzIEFuYWx5c3QgSUk8L2E-PC90ZD4NCiAgICAgICAgICAgICAgICA8L3RyPg0KICAgICAgICAgICAgDQogICAgICAgICAgICAgICAgPHRyPg0KICAgICAgICAgICAgICAgICAgICA8dGQ-PGEgaHJlZj0naHR0cHM6Ly9jbGV2ZWxhbmRjbGluaWMub3JnLyc-Q2xldmVsYW5kIENsaW5pYzwvYT48L3RkPg0KICAgICAgICAgICAgICAgICAgICA8dGQ-PGEgaHJlZj0naHR0cHM6Ly9jY2Yud2QxLm15d29ya2RheWpvYnMuY29tL0NsZXZlbGFuZENsaW5pY0NhcmVlcnMvam9iL1JlbW90ZS1Mb2NhdGlvbi9TeXN0ZW0tQW5hbHlzdHMtSUlJLS1FcGljLVByZWx1ZGUtY2VydGlmaWNhdGlvbi1vci1FcGljLXJlZmVycmFsLWF1dGhvcml6YXRpb24tY2VydGlmaWNhdGlvbi1yZXF1aXJlZC1fMjY0ODE2Jz5TeXN0ZW0gQW5hbHlzdHMgSUlJPC9hPjwvdGQ-DQogICAgICAgICAgICAgICAgPC90cj4NCiAgICAgICAgICAgIA0KICAgICAgICAgICAgICAgIDx0cj4NCiAgICAgICAgICAgICAgICAgICAgPHRkPjxhIGhyZWY9J2h0dHBzOi8vd3d3LmNyb3dkc3RyaWtlLmNvbS8nPkNyb3dkU3RyaWtlPC9hPjwvdGQ-DQogICAgICAgICAgICAgICAgICAgIDx0ZD48YSBocmVmPSdodHRwczovL2Nyb3dkc3RyaWtlLndkNS5teXdvcmtkYXlqb2JzLmNvbS9jcm93ZHN0cmlrZWNhcmVlcnMvam9iL1VTQS0tLVJlbW90ZS9TaXRlLVJlbGlhYmlsaXR5LUVuZ2luZWVyLS1SZW1vdGUtX1IxOTAyOS0xJz5TaXRlIFJlbGlhYmlsaXR5IEVuZ2luZWVyIChSZW1vdGUpPC9hPjwvdGQ-DQogICAgICAgICAgICAgICAgPC90cj4NCiAgICAgICAgICAgIA0KICAgICAgICAgICAgICAgIDx0cj4NCiAgICAgICAgICAgICAgICAgICAgPHRkPjxhIGhyZWY9J2h0dHBzOi8vc3dlZGlzaGNvdmVuYW50Lm9yZy8nPlN3ZWRpc2ggQ292ZW5hbnQgSGVhbHRoPC9hPjwvdGQ-DQogICAgICAgICAgICAgICAgICAgIDx0ZD48YSBocmVmPSdodHRwczovL25zaHMud2QxLm15d29ya2RheWpvYnMuY29tL25zLWVlaC9qb2IvTkNILTg4MC1XLUNlbnRyYWwtUmQtQXJsaW5ndG9uLUhlaWdodHMvRGF0YS1BbmFseXN0LS0tTmV1cm9zY2llbmNlc19SMTY2ODQnPkRhdGEgQW5hbHlzdCAtIE5ldXJvc2NpZW5jZXM8L2E-PC90ZD4NCiAgICAgICAgICAgICAgICA8L3RyPg0KICAgICAgICAgICAgDQogICAgICAgICAgICAgICAgPHRyPg0KICAgICAgICAgICAgICAgICAgICA8dGQ-PGEgaHJlZj0naHR0cHM6Ly93d3cuY2l0aWdyb3VwLmNvbS8nPkNpdGk8L2E-PC90ZD4NCiAgICAgICAgICAgICAgICAgICAgPHRkPjxhIGhyZWY9J2h0dHBzOi8vY2l0aS53ZDUubXl3b3JrZGF5am9icy5jb20vMi9qb2IvQ2l1ZGFkLURlLU1leGljby1EaXN0cml0by1GZWRlcmFsLU1leGljby9JT1MtRGV2ZWxvcGVyLU1vYmlsZS1BcHBfMjQ3NjkwNDAnPklPUyBEZXZlbG9wZXIgTW9iaWxlIEFwcDwvYT48L3RkPg0KICAgICAgICAgICAgICAgIDwvdHI-DQogICAgICAgICAgICANCiAgICAgICAgICAgICAgICA8dHI-DQogICAgICAgICAgICAgICAgICAgIDx0ZD48YSBocmVmPSdodHRwczovL2pvYnMuZGlzbmV5Y2FyZWVycy5jb20vJz5EaXNuZXkgQ2FyZWVyczwvYT48L3RkPg0KICAgICAgICAgICAgICAgICAgICA8dGQ-PGEgaHJlZj0naHR0cHM6Ly9kaXNuZXkud2Q1Lm15d29ya2RheWpvYnMuY29tL2Rpc25leWNhcmVlci9qb2IvQ2hhcmxvdHRlLU5DLVVTQS9NZWRpYS1FbmdpbmVlci1JSV8xMDA5MDE2OCc-TWVkaWEgRW5naW5lZXIgSUk8L2E-PC90ZD4NCiAgICAgICAgICAgICAgICA8L3RyPg0KICAgICAgICAgICAgDQogICAgICAgICAgICAgICAgPHRyPg0KICAgICAgICAgICAgICAgICAgICA8dGQ-PGEgaHJlZj0naHR0cHM6Ly9ha3VuYWNhcGl0YWwuY29tLyc-QWt1bmEgQ2FwaXRhbDwvYT48L3RkPg0KICAgICAgICAgICAgICAgICAgICA8dGQ-PGEgaHJlZj0naHR0cHM6Ly93d3cuYWt1bmFjYXBpdGFsLmNvbS9qb2ItZGV0YWlscz9naF9qaWQ9NTc0NDI2Nyc-SnVuaW9yIFF1YW50aXRhdGl2ZSBSZXNlYXJjaGVyPC9hPjwvdGQ-DQogICAgICAgICAgICAgICAgPC90cj4NCiAgICAgICAgICAgIA0KICAgICAgICAgICAgICAgIDx0cj4NCiAgICAgICAgICAgICAgICAgICAgPHRkPjxhIGhyZWY9J2h0dHBzOi8vd3d3LmFzc2VtYmx5YWkuY29tLyc-QXNzZW1ibHlBSTwvYT48L3RkPg0KICAgICAgICAgICAgICAgICAgICA8dGQ-PGEgaHJlZj0naHR0cHM6Ly9qb2ItYm9hcmRzLmdyZWVuaG91c2UuaW8vYXNzZW1ibHlhaS9qb2JzLzQ0NDMyNDQwMDUnPlNvZnR3YXJlIEVuZ2luZWVyIC0gRnVsbHN0YWNrL0Zyb250ZW5kPC9hPjwvdGQ-DQogICAgICAgICAgICAgICAgPC90cj4NCiAgICAgICAgICAgIA0KICAgICAgICAgICAgICAgIDx0cj4NCiAgICAgICAgICAgICAgICAgICAgPHRkPjxhIGhyZWY9J2h0dHBzOi8vd3d3LmJvZWluZy5jb20vJz5Cb2Vpbmc8L2E-PC90ZD4NCiAgICAgICAgICAgICAgICAgICAgPHRkPjxhIGhyZWY9J2h0dHBzOi8vYm9laW5nLndkMS5teXdvcmtkYXlqb2JzLmNvbS9FWFRFUk5BTF9DQVJFRVJTL2pvYi9VU0EtLS1IYXplbHdvb2QtTU8vTWlkLUxldmVsLVNvZnR3YXJlLUVuZ2luZWVyXzAwMDAwNDMwNTQzLTInPk1pZC1MZXZlbCBTb2Z0d2FyZSBFbmdpbmVlcjwvYT48L3RkPg0KICAgICAgICAgICAgICAgIDwvdHI-DQogICAgICAgICAgICA8L3RhYmxlPg0KICAgICAgICA8YnI-DQogICAgICAgIDxwPjxhIGhyZWY9J2h0dHBzOi8vYXBwLm5vdGlmeS5jYXJlZXJzJz5ub3RpZnkuY2FyZWVyczwvYT48L3A-DQogICAgICAgIDxwPjxhIGhyZWY9J2h0dHBzOi8vYXBwLm5vdGlmeS5jYXJlZXJzL25vdGlmaWNhdGlvbnMnPnVwZGF0ZSBlbWFpbCBub3RpZmljYXRpb25zPC9hPjwvcD4NCiAgICAgICAgPC9ib2R5Pg0KICAgICAgICA8L2h0bWw-DQogICAgDQo=")
soup = BeautifulSoup(decoded_data , "html.parser")
body = soup.get_text()

body

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


"\n\nCompanyJob\n\n\n\n\nCrowdStrike\n\n\nDisney Careers\n\n\n\nMid-Level Software Engineer\r\n                </tr\x03B�\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x0f\x0b�\x18X�\x19O�B�\x08\x08\x08\x08\x08\x08\x08\x0f\x18��Т\x02\x02\x02\x02\x02\x02\x02\x03�\x03��\x12\x06�&Vc�v�GG\x073���\x17\x07\x02���F�g��6\x17&VW'2s���F�g��6\x17&VW'3��\x13���\x004(�����������\U00044061ɕ��������輽������ѥ�乍�ɕ��̽��ѥ����ѥ��̜�����є���������ѥ����ѥ��������4(��������\U0003d27d���4(���������ѵ�\r\n    \r\n"

In [184]:
import classification as const
for x in const.Classifications:
    print(x)

Job Applied Confirmation
Job Rejection
Job Offered
Completed task corresponding to a Job
New Job Notification
Job Task or To Do
Not Job Specific Email


In [179]:
import os
import openai
from dotenv import load_dotenv

load_dotenv()
pinecone_api_key = os.getenv("PINECONE_API_KEY")
openai.api_key = os.getenv("OPENAI_API_KEY")

In [220]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# Prompt
prompt_text = """You are an assistant tasked with summarizing and classifying emails. 

First, summarize the email content concisely and only include the links present corresponding to jobs in the email. Then, classify the email as one of the following categories based on its content. Use the criteria provided for each category:

1. **Job Applied Confirmation**: Emails confirming that a job application has been successfully received or submitted. These emails often thank the applicant for their interest in the position and provide further instructions or timelines for the next steps.

2. **Job Rejection**: Emails that inform the recipient that their application has not been successful. These emails usually contain phrases like "we regret to inform you," "we have decided to move forward with other candidates," or similar language indicating a rejection.

3. **Job Offered**: Emails that offer a job to the recipient. They often include details about the job offer, such as position title, salary, benefits, and instructions for how to accept the offer.

4. **Completed Task Corresponding to a Job**: Emails that confirm the completion of a task related to a job application or interview process, such as the submission of a coding test, assessment, or any required documents. If an email mentions that a test has been submitted or a task is completed for a job application, classify it as this.

5. **New Job Notification**: Emails that notify the recipient about new job openings or opportunities. These might come from job boards, recruiters, or companies and often include descriptions of available positions.

6. **Job Task or To Do**: Emails that include tasks or action items related to a job, such as follow-up actions after an interview, requests for additional information, or other job-related tasks that need to be completed by the recipient.

7. **Not Job Specific Email**: Emails that do not pertain to a specific job application, job offer, or job-related task. These could be newsletters, general information, or any emails not directly related to a job process.

Email: {element}

Output:
Summary: [Provide the summary here]
Classification: [Provide the classification here]
"""
prompt = ChatPromptTemplate.from_template(prompt_text)

# Summary chain
model = ChatOpenAI(temperature=0, model="gpt-4o")
summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()
summarize_chain

{
  element: RunnableLambda(lambda x: x)
}
| ChatPromptTemplate(input_variables=['element'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['element'], template='You are an assistant tasked with summarizing and classifying emails. \n\nFirst, summarize the email content concisely and only include the links present corresponding to jobs in the email. Then, classify the email as one of the following categories based on its content. Use the criteria provided for each category:\n\n1. **Job Applied Confirmation**: Emails confirming that a job application has been successfully received or submitted. These emails often thank the applicant for their interest in the position and provide further instructions or timelines for the next steps.\n\n2. **Job Rejection**: Emails that inform the recipient that their application has not been successful. These emails usually contain phrases like "we regret to inform you," "we have decided to move forward with other candidates,"

In [221]:
import classification as const

def classify_email(email_content):
    prompt_text1 = f"""You are a job email classifier. I want you to classify this email as one of the following: {', '.join(const.Classifications)}, just return the classification values. Here is the email: {email_content}. Use the criteria provided for each category:

    1. **Job Applied Confirmation**: Emails confirming that a job application has been successfully received or submitted. These emails often thank the applicant for their interest in the position and provide further instructions or timelines for the next steps.

    2. **Job Rejection**: Emails that inform the recipient that their application has not been successful. These emails usually contain phrases like "we regret to inform you," "we have decided to move forward with other candidates," or similar language indicating a rejection.

    3. **Job Offered**: Emails that offer a job to the recipient. They often include details about the job offer, such as position title, salary, benefits, and instructions for how to accept the offer.

    4. **Completed Task Corresponding to a Job**: Emails that confirm the completion of a task related to a job application or interview process, such as the submission of a coding test, assessment, or any required documents. If an email mentions that a test has been submitted or a task is completed for a job application, classify it as this.

    5. **New Job Notification**: Emails that notify the recipient about new job openings or opportunities. These might come from job boards, recruiters, or companies and often include descriptions of available positions.

    6. **Job Task or To Do**: Emails that include tasks or action items related to a job, such as follow-up actions after an interview, requests for additional information, or other job-related tasks that need to be completed by the recipient.

    7. **Not Job Specific Email**: Emails that do not pertain to a specific job application, job offer, or job-related task. These could be newsletters, general information, or any emails not directly related to a job process."""
    
    response = model(prompt_text1)
    return response.content

texts=[]
email_summaries = []
email_classifications = []
for filename in os.listdir("emails"):
		if filename.endswith(".txt"):
			filepath = os.path.join("emails", filename)
			with open(filepath, 'r', encoding='utf-8') as file:
				email_content = file.read()
				# Get the summary
				texts.append(email_content)
				input_data = {"element": email_content}
				summary = summarize_chain.invoke(input_data)
				email_summaries.append(summary)
				classification = classify_email(email_content)
				email_classifications.append(classification)

				

In [222]:
email_summaries

['**Summary:**\nThe email lists new job postings with links to the respective job applications:\n- [Associate Software Engineer at Metron](https://boards.greenhouse.io/metron/jobs/4495423007)\n- [Data Acquisition Standards Associate at IQVIA Inc](https://iqvia.wd1.myworkdayjobs.com/IQVIA/job/Durham-North-Carolina-United-States-of-America/Data-Acquisition-Standards-Associate_R1439570)\n- [Intern, myQ Front End Developer (Android or iOS) (Summer 2025) at Chamberlain Group](https://chamberlain.wd1.myworkdayjobs.com/Chamberlain_Group/job/Oak-Brook-IL/Intern--myQ-Front-End-Developer--Android-or-iOS---Summer-2025-_JR27802)\n- [Intern, Machine Learning, Computer Vision, and AI (Summer 2025) at Chamberlain Group](https://chamberlain.wd1.myworkdayjobs.com/Chamberlain_Group/job/Oak-Brook-IL/Intern--Machine-Learning--Computer-Vision--and-AI--Summer-2025-_JR27785)\n- [Data Scientist 2 at PayPal](https://wd1.myworkdaysite.com/recruiting/paypal/jobs/job/California-United-States-of-America-Virtual/Da

In [223]:
import uuid
import pinecone
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryStore
from langchain_pinecone import PineconeVectorStore
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import VectorStore, VectorStoreRetriever



import os
from pinecone import Pinecone, ServerlessSpec, PodSpec

pc = Pinecone(
    api_key= pinecone_api_key
)

# Create the index and populate it
om_index = 'om-test-1'
if om_index not in pc.list_indexes().names():
    pc.create_index(
        name=om_index,
        dimension=1536,
        metric='cosine',
        spec=ServerlessSpec(
          cloud = 'aws',
          region = "us-east-1")
    )

## Uploading the documents to the vector database
vectorstore = PineconeVectorStore(
    embedding=OpenAIEmbeddings(),
    index_name=om_index
)

doc_ids = [str(uuid.uuid4()) for _ in texts]
metadata = [{"classification": x} for x in email_classifications]
summary_texts = [
    Document(page_content=s, metadata={"doc_id": doc_ids[i], "classification": email_classifications[i]})
    for i, s in enumerate(email_summaries)
]

# Add documents to the vector store
vectorstore.add_documents(documents=summary_texts)

['7e3723af-cff0-4f8c-8aed-9dadc9c02add',
 'fb5c9467-d5ed-4975-b3e6-43ff5580daf3',
 'b0548da4-2b61-4a77-8871-c63886bbee0b',
 'ad441b11-56cf-45d0-a03f-153f001bf3a0',
 '8b2bad7f-69b9-4212-9292-0e140245e982',
 'aca0c803-95e9-4b39-a192-1ab64d319639',
 '4971b561-7e8f-42a5-ade8-a729689ca842',
 '6acef088-2d85-42cb-82f4-48b5da48d7ad',
 '94d32672-bd91-4aa7-aa14-ce3852b073d6',
 '4ef38edc-92de-41bd-af99-9d69be65a3bc',
 'eec4c6b3-2973-4a3c-8fc9-d2318566c836',
 'a7e394c4-87b4-4dc0-bc6c-8b1b90fc57a7',
 '5964525d-d781-4a37-88d4-404952766ebc',
 '6f6c159a-4a25-46f7-9733-4833002de32d',
 'a6f6b97d-df66-4923-907c-67781cffef9f',
 '34873ae1-a7f7-4f80-a311-2c6568d1934c',
 '23ebc21b-e5fe-44ed-a4f3-f65f37d747f4',
 '9901175f-b69a-4d74-bb2b-09aee52ac8da',
 'feea5793-1c5d-4360-a15c-affdc0d0db4a',
 '74ffdf70-1508-4f77-a649-2aa57d48425c',
 '84c05166-dfdb-4eef-a5fd-92987922e954',
 '7a23a610-86e3-4c75-94a7-f5cec016dd01',
 'dce3c3c2-01d9-4622-a43f-66781be1c653',
 'c206c5d9-71b9-4a56-a5b0-90dc2d7cfafb',
 '7737d094-b2e9-

In [189]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 15, "filter": {"classification": "Job Task or To Do"}})

In [224]:
def retrieve_context_for_classification(classification):
    retriever = vectorstore.as_retriever(
        search_kwargs={"k": 100, "filter": {"classification": classification}}
    )
    retrieved_docs = retriever.get_relevant_documents(classification)
    context = "\n".join([doc.page_content for doc in retrieved_docs])
    return context


from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter

# Prompt template
template = """Answer the question based only on the following context, which include email contents:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# LLM
model = ChatOpenAI(temperature=0, model="gpt-4o")

# RAG pipeline
chain = (
    {
        "context": lambda inputs: retrieve_context_for_classification(inputs["classification"]),
        "question": RunnablePassthrough()
    }
    | prompt
    | model
    | StrOutputParser()
)

In [173]:
print(chain.invoke({
    "query": "New Job Notification",
    "question": "You are a email monitor agent, and you need to make a list of all the job roles with the company name and if same job is found in multiple emails just put it once in the list."
}))

1. Software Engineer - Stripe (Canada)
2. Technology Software Engineer Summer Internship - American Express
3. Software Application Developer - Officer (Onsite) - State Street (Massachusetts, USA)
4. Software Development Engineer - Sr. Associate-6 - State Street (Massachusetts, USA)
5. Sr. Software Engineer - State Street (Massachusetts, USA)
6. Senior Software Engineer - State Street (Massachusetts, USA)
7. Backend Engineer - Ranking Foundation - Reddit
8. Intern, Data Science and Analytics - Fannie Mae (Flexible Hybrid)
9. Intern, Data Science - Wealthsimple (Fall 2024)
10. Data Engineer - FedEx Dataworks (USA)
11. Data Engineer - Capgemini (USA)
12. Data Engineer - SynergisticIT (USA)
13. Data Engineer - Talution Group (USA)
14. Data Engineer - CyberCoders (USA)
15. Data Engineer - Motion Recruitment (USA)
16. Data Scientist - Algorized (USA)
17. Data Scientist - Govini (USA)
18. Data Scientist - Coinbase (USA)
19. Data Scientist - Capital One (USA)
20. Data Scientist - L.E.K. Consu

In [225]:
classification_questions = {
    "Job Applied Confirmation": "You are an email monitor agent, and you need to make a list of all companies where a job application has been confirmed in the format [Company Name][Job Title] and if the Job Title is not mentioned put SWE role.",
    "Job Rejection": "You are an email monitor agent, and you need to make a list of all companies that have sent job rejection emails in the format [Company Name][Job Title] and if the Job Title is not mentioned put SWE role.",
    "Job Offered": "You are an email monitor agent, and you need to make a list of all companies that have a sent a job offer email in the format [Company Name][Job Title] and if the Job Title is not mentioned put SWE role.",
    "Completed Task Corresponding to a Job": "You are an email monitor agent, and you need to make a list of all completed tasks corresponding to a job application emails in the format [Company Name][Job Title] and if the Job Title is not mentioned put SWE role.",
    "New Job Notification": "You are an email monitor agent, and you need to make a list of all new job notifications received in the format [Company Name][Job Title][Link] and if the same job is found in multiple emails just put it once in the list.",
    "Job Task or To Do": "You are an email monitor agent, and you need to make a list of all tasks or to-dos related to job applications emails in the format [Company Name][Job Title] and if the Job Title is not mentioned put SWE role.",
    "Not Job Specific Email": "List all emails not specific to any job application."
}

In [226]:
results = {}
for classification, question in classification_questions.items():
    print(f"Processing classification: {classification}")
    context = retrieve_context_for_classification(classification)
    if context.strip():  # Check if context is not empty
        result = chain.invoke({
            "classification": classification,
            "question": question
        })
    else:
        result = "No emails associated with this classification category."
    results[classification] = result
    print(f"Result for {classification}: {result}\n")

# Output all results
for classification, result in results.items():
    print(f"Classification: {classification}\nResult: {result}\n")

Processing classification: Job Applied Confirmation
Result for Job Applied Confirmation: 1. [Intel][SWE role]
2. [Adobe][2025 University Graduate - Software Engineer]
3. [Disney][Computer Science & Computer Engineering Intern, Spring 2025]

Processing classification: Job Rejection
Result for Job Rejection: 1. Alignable [Software Engineering Co-Op]
2. Nokia [Software Engineering Co-op]
3. Nokia [UAV Software Engineering Co-op]
4. Disney [Computer Science & Computer Engineering Intern, Spring 2025]

Processing classification: Job Offered
Result for Job Offered: No emails associated with this classification category.

Processing classification: Completed Task Corresponding to a Job
Result for Completed Task Corresponding to a Job: 1. [Adobe][2025 University Graduate Software Engineer (R147863)]
2. [Adobe][2025 University Graduate Software Engineer (R147900)]
3. [Adobe][2025 University Graduate Software Engineer (R147902)]

Processing classification: New Job Notification
Result for New Job

In [164]:
retrieved_docs

[Document(metadata={'classification': 'Completed task corresponding to a Job', 'doc_id': '4a5a1e1c-f267-483f-b003-a5e245af37d1'}, page_content="Summary: The email confirms the recipient's completion and submission of the SWE1 C++ Test 2025 - Round 1 to Akuna Capital. It encourages the recipient to continue practicing coding challenges on HackerRank and provides a reminder that any follow-up questions should be directed to Akuna Capital.\n\nClassification: Completed Task Corresponding to a Job"),
 Document(metadata={'classification': 'Job Applied Confirmation', 'doc_id': '9ff9a1eb-07da-4f3d-bab7-12958ce2b615'}, page_content="Summary: The email confirms that Vinit's application for the Machine Learning Developer Intern position at Revent has been successfully sent. It also suggests other job opportunities that Vinit might be interested in, including positions at Nextdoor and Semgrep.\n\nClassification: Job Applied Confirmation"),
 Document(metadata={'classification': 'Job Applied Confirm